In [ ]:
import numpy as np     # get ODE solvers, numpy
import vpython as vp         # get VPython modules for animation
vec=vp.vector

def leapfrog(lfdiffeq, r1, v1, r2, h, GM):       # vectorized leapfrog
    """ vector leapfrog method using numpy arrays.
        It solves general (r,v) ODEs as: 
        dr[i]/dt = f[i](v), and dv[i]/dt = g[i](r).
        User supplied lfdiffeq(id, r, v, t) returns
        f[i](r) if id=0, or g[i](v) if id=1.
        It must return a numpy array if i>1 """
    hh = h/2.0
    r11 = r1 + hh*lfdiffeq(0, r1, v1, r2, GM)     # 1st: r at h/2 using v0    @\lbl{line:lf1}@
    v11 = v1 +  h*lfdiffeq(1, r11, v1, r2, GM)  # 2nd: v1 using a(r) at h/2 @\lbl{line:lf2}@
    r11 = r11 + hh*lfdiffeq(0, r1, v11, r2, GM)   # 3rd: r1 at h using v1     @\lbl{line:lf3}@
    return r1, v1
  
     
def velo(id, r1, v1, r2, GM):            # return the eqns of motion
    if (id == 0): return v1        # velocity, dr/dt
    r12 = r2 - r1
    s = vp.mag(vec(r12[0],r12[1],0))   # $s=|\vec{r}|$
    return GM*r12/(s*s*s)           # accel dv/dt, faster than s**3  
        
def go():
    r_e = np.array([1.4960, 0.0])*(10**11)    # initial x,y position for earth  
    r_s = np.array([0.0, 0.0])   # initial x,y position for sun   
    v_e = np.array([0.0, 29.8])*(10**3)    # initial vx, vy  for earth 
    v_s = np.array([0.0, 0.0])    # initial vx, vy  for sun 
    
    # draw the scene, planet earth/path, sun/sunlight               
    scene = vp.canvas(title='Planetary motion',          # scene start 
                       background=vec(.2,.5,1), forward=vec(0,2,-1))
    earth= vp.sphere(pos=vec(r_e[0],r_e[1],0), radius=6371000*1000, make_trail=True, up=vec(0,0,1))
    sun   = vp.sphere(pos=vec(r_s[0],r_s[1],0), radius=695508000*10, make_trail=True, color=vp.color.yellow)
    sunlight = vp.local_light(pos=vec(0,0,0), color=vp.color.yellow) #scn end 
    
    t, h = 0.0, 0.001
    while True:
       # vp.rate(10)   # limit animation speed
        r_e, v_e = leapfrog(velo, r_e, v_e, r_s, h, m_s)  # integrate 
        earth.pos = vec(r_e[0],r_e[1],0)           # move earth    
        r_s, v_s = leapfrog(velo, r_s, v_s, r_e, h, m_e)  # integrate 
        sun.pos = vec(r_s[0],r_s[1],0)           # move sun  

        
m_s = 1.989* 10**30
m_e = 5.972 * 10**24
go()
       
